In [1]:
from vietadminunits import parse_address, get_data
import pandas as pd
import re

from unidecode import unidecode

# Demo

In [ ]:
parse_address('ky anh ha tinh')

In [ ]:
parse_address('hat lot mai son son la')

In [ ]:
parse_address('phuong Tan Thanh, Ca Mau, Ca Mau')

In [ ]:
parse_address('Ngã Tư Sở Market, Đường Nguyễn Trãi, Thượng Đình Số nhà 28A, Ngõ 72 Nguyễn Trãi , , Phường Thượng Đình, Quận Thanh Xuân , Hà Nội Quận Thanh Xuân')

In [ ]:
parse_address('Cuối đường kiều mai, dự án bệnh viện hạnh phúc  phường phú diễn quận bắc từ liêm hà nội')

In [ ]:
parse_address('03 Nguyễn Cao Luyện, An Hải Bắc, Sơn Trà, Đà Nẵng 550000, Việt Nam')

In [ ]:
parse_address('3c60/10 chợ cầu xấng, xã phạm văn hai, huyện bình chánh HCM')

In [ ]:
parse_address('ngõ 154 Đình Thôn, phường Mỹ Đình, Quận Nam Từ Liêm, HN')

In [ ]:
parse_address('ngõ 154 Đình Thôn, phường Mỹ Đình, Quận Nam Từ Liêm, HN', level=1)

In [ ]:
parse_address('Ha Noi, Gia Lam, Yen Vien')

# Check wrong or missing in bulk

In [ ]:
df = pd.read_csv('../data/input/danhmuchanhchinhgso.gov.vn.csv')

In [ ]:
df['district_address'] = df['long_district'] + ', ' + df['long_province']
district_addresses = df['district_address'].tolist()

for address in district_addresses:
    try:
        unit = parse_address(address, level=2)
    except:
        print(address)
    if unit.province == None:
        print(address)
    elif unit.district == None:
        print(address)

In [ ]:
df['ward_address'] = df['long_ward'].fillna('') + ', ' + df['long_district'] + ', ' + df['long_province']
ward_addresses = df['ward_address'].dropna().tolist()
miss = []
for ward_address in ward_addresses:
    unit = parse_address(ward_address, level=3)

    if unit.province == None:
        print(ward_address)
    elif unit.district == None:
        print(ward_address)
    elif unit.ward == None and unit.district_key not in ['bachlongvi', 'conco', 'hoangsa', 'lyson', 'condao']:
        print(ward_address)
